In [1]:
import sqlite3
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
##create the connection with sqllite 3 database
con= sqlite3.connect('database.sqlite')
#Create DatFrame
df = pd.read_sql_query("SELECT * FROM Player_Attributes", con)

In [3]:
df.head(3)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0


In [4]:
df.shape

(183978, 42)

In [5]:
#FOR exploring the dataset
df.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


In [6]:
#for removing the null values from the  dataframe
df = df.dropna(how='any',axis=0) 

In [7]:
df.shape

(180354, 42)

In [8]:
df.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,...,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.000000,180354.00000
mean,91995.886274,166822.125803,137653.145514,68.635317,73.479457,55.142071,49.962136,57.263476,62.486726,49.488927,...,57.868176,54.933448,46.757433,50.354065,48.029342,14.696685,15.947786,20.526304,16.015043,16.32531
std,53092.657914,52821.443279,137599.735284,7.027950,6.581963,17.247231,19.041760,16.478716,14.172493,18.252319,...,15.152408,15.556645,21.226730,21.496289,21.592830,16.841454,15.841297,21.143898,16.070772,17.18545
min,1.000000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000
25%,46074.250000,156616.000000,35451.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.00000
50%,92003.500000,183792.000000,80291.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.00000
75%,137935.750000,200138.000000,192841.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.00000
max,183978.000000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,94.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.00000


In [9]:
df1=df  #Saving our origninal dataframe into otherdataframe for datapreprocessing

In [10]:
df1=df1[df1.columns[df1.dtypes=='object']] #finding features of type object

In [11]:
df1.columns

Index(['date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], dtype='object')

In [12]:
df1.head()

,date,preferred_foot,attacking_work_rate,defensive_work_rate
0,2016-02-18 00:00:00,right,medium,medium
1,2015-11-19 00:00:00,right,medium,medium
2,2015-09-21 00:00:00,right,medium,medium
3,2015-03-20 00:00:00,right,medium,medium
4,2007-02-22 00:00:00,right,medium,medium


In [13]:
df1['date']=df1['date'].astype('datetime64[ns]')  #converting the datadatatype from string

C:\Users\Abhi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
df1.head(3)

,date,preferred_foot,attacking_work_rate,defensive_work_rate
0,2016-02-18,right,medium,medium
1,2015-11-19,right,medium,medium
2,2015-09-21,right,medium,medium


In [15]:
# now converting object values to categorical values 

In [16]:
from sklearn.preprocessing  import OrdinalEncoder

In [17]:
cat=OrdinalEncoder()

In [18]:
#Since the date is not categorical we will be removing this thing 

In [19]:
#df1=df1[1:] or
df1=df1.drop(columns='date')

In [20]:
#now we will convert these categorical columns into numerical one

In [21]:
for i in df1.columns:
    df1[i]=cat.fit_transform(df1[i].values.reshape(-1,1))

In [22]:
df1.head(3)

,preferred_foot,attacking_work_rate,defensive_work_rate
0,1.0,4.0,14.0
1,1.0,4.0,14.0
2,1.0,4.0,14.0


In [23]:
#now we wiil drop the date column and the categorical columns in df ans replace them by the df1 columns

In [24]:
df=df.drop(columns='date')
df.head(2)

,id,player_fifa_api_id,player_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,67.0,71.0,right,medium,medium,49.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,67.0,71.0,right,medium,medium,49.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


In [25]:
df=df.drop(columns=['preferred_foot','attacking_work_rate','defensive_work_rate'])
df.head(3)

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,67.0,71.0,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,67.0,71.0,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,62.0,66.0,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0


In [26]:
df.shape

(180354, 38)

In [27]:
for i in df1.columns:
    df.insert(loc=len(df.columns),value=df1[i],column=df1[i].name)
  

In [28]:
df.shape

(180354, 41)

In [29]:
df.head()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,1,218353,505942,67.0,71.0,49.0,44.0,71.0,61.0,44.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
1,2,218353,505942,67.0,71.0,49.0,44.0,71.0,61.0,44.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
2,3,218353,505942,62.0,66.0,49.0,44.0,71.0,61.0,44.0,...,66.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
3,4,218353,505942,61.0,65.0,48.0,43.0,70.0,60.0,43.0,...,63.0,66.0,5.0,10.0,9.0,7.0,7.0,1.0,4.0,14.0
4,5,218353,505942,61.0,65.0,48.0,43.0,70.0,60.0,43.0,...,63.0,66.0,5.0,10.0,9.0,7.0,7.0,1.0,4.0,14.0


In [30]:
#now the following colums are just the id of player and play no role in determining the player ranking 
col= ['id','player_fifa_api_id','player_api_id']
#dropping those columns from the dataset
df=df.drop(columns=col)
df.head(3)

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
1,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
2,62.0,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,66.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0


In [31]:
df.shape

(180354, 38)

In [32]:
#now overall rating  is the target variable so spliting it from the dataframe df
df_X=df.iloc[:,1:]

In [33]:
df_X.shape

(180354, 37)

In [34]:
df_X.head()

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
1,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,...,69.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
2,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,...,66.0,69.0,6.0,11.0,10.0,8.0,8.0,1.0,4.0,14.0
3,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,...,63.0,66.0,5.0,10.0,9.0,7.0,7.0,1.0,4.0,14.0
4,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,...,63.0,66.0,5.0,10.0,9.0,7.0,7.0,1.0,4.0,14.0


In [35]:
df_Y=df.iloc[:,:1]

In [36]:
df_Y.head()

,overall_rating
0,67.0
1,67.0
2,62.0
3,61.0
4,61.0


In [37]:
df_Y.shape

(180354, 1)

In [38]:
#now spliting the data inot traingi and test data

In [39]:
df_X_train,df_X_test,df_Y_train,df_Y_test=train_test_split(df_X,df_Y,test_size=0.30,random_state=5)

In [40]:
df_X_train.shape

(126247, 37)

In [41]:
df_X_test.shape

(54107, 37)

In [42]:
df_Y_train.shape

(126247, 1)

In [43]:
df_Y_test.shape

(54107, 1)

In [44]:
#now train the model on training data

In [45]:
reg=LinearRegression()

In [46]:
reg.fit(df_X_train,df_Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [47]:
reg.intercept_    #intercept for the slope of the line that is best fit

array([-2.49825705])

In [48]:
reg.coef_   #cefficients for the weight of features for which line is best fit

array([[ 0.37602698,  0.02234804,  0.01077481,  0.07045282,  0.05065366,
         0.00446231, -0.01349775,  0.0105653 ,  0.01329217,  0.00633646,
         0.1354272 ,  0.00607505,  0.00890324, -0.00894099,  0.21279691,
         0.00805585,  0.01654217,  0.01492055, -0.00533128,  0.0606972 ,
        -0.01318302,  0.02005851,  0.01310308, -0.01053269, -0.00221224,
         0.01404741,  0.0316956 ,  0.00317027, -0.02866742,  0.16116261,
         0.0365576 , -0.0343538 ,  0.05532635,  0.02462744, -0.00144177,
        -0.03769678, -0.05770691]])

In [49]:
reg.coef_ .shape

(1, 37)

In [50]:
#to claculate the squared error distance    
import numpy as np
dist=np.mean((reg.predict(df_X_test)-df_Y_test)**2)

In [51]:
dist

overall_rating    7.796222
dtype: float64

In [52]:
#To calculate the accuracy of the model
print("Accuracy score is %.3f"%reg.score(df_X_test,df_Y_test))

Accuracy score is 0.842


In [65]:
#Let's Calculate the actual Square Root  mean of distance from  the Predicted values andthe actual values
from sklearn.metrics import mean_squared_error


In [66]:
rmse=np.sqrt(np.sum((reg.predict(df_X_test)-df_Y_test)**2)/len(df_Y_test))

In [67]:
rmse

overall_rating    2.792172
dtype: float64

In [68]:
# Check the Accuracy of the model using cross validation 

In [69]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(reg, df_X_test, df_Y_test, cv=10)
tree_rmse_scores = np.sqrt(scores)


In [70]:
tree_rmse_scores

array([0.91626786, 0.91811274, 0.91771093, 0.91701574, 0.91713954,
       0.92002708, 0.92162603, 0.91459115, 0.91935193, 0.91547093])

In [71]:
print(scores)
print(np.mean(scores))
print(np.std(scores))

[0.83954678 0.84293101 0.84219334 0.84091787 0.84114493 0.84644982
 0.84939454 0.83647697 0.84520797 0.83808702]
0.842235026258991
0.003726374550491807
